### Encoding de variables categóricas

In [1]:
import numpy as np
import pandas as pd
# Gráficos
# ------------------------------------------------------------------------------
import matplotlib.pyplot as plt
import seaborn as sns
# Modelado y evaluación
# ------------------------------------------------------------------------------
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_val_score
from sklearn import tree
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, cohen_kappa_score
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import GridSearchCV
# Configuración warnings
# ------------------------------------------------------------------------------
import warnings
warnings.filterwarnings("ignore")

In [2]:
df = pd.read_pickle("../data/bikes_registrados_estand.pkl")
df.head(2)

,fecha,estacion,año,mes,festivo,dia,laborable,meteo,temperatura,humedad,velo_viento,usu_registrados
0,2018-01-01,invierno,2018,ene,festivo,lun,no laborable,nublado,-0.486274,0.858854,-0.208909,654
1,2018-01-02,invierno,2018,ene,no festivo,mar,laborable,nublado,-0.425690,0.331548,0.687550,670


In [3]:
# Vamos a eliminar las variables categóricas "estación" y "laborable"  porque las creamos a partir del resto de variables predictoras y, además, 
# ya comprobamos en la visualización que tenían una relación muy alta con las otras VP. Eliminamos también fecha porque tiene un valor por cada registro
df.drop(["laborable", "estacion", "fecha"], axis = 1, inplace= True)

Según vimos en "visualización", todas nuestras variables predictorias categóricas tienen orden, por lo que vamos a usar el "map" (para declarar nosotras mismas el orden de las variables):

In [4]:
categoricas = df.select_dtypes(include=["O", "category"])
categoricas.columns

Index(['año', 'mes', 'festivo', 'dia', 'meteo'], dtype='object')

In [5]:
columnas = ["mes", "festivo", "dia", "meteo", "año"]

mes = {"ene": 0, 
       "feb": 1, 
       "mar": 2, 
       "abr": 3, 
       "may": 4, 
       "jun": 6, 
       "jul": 5, 
       "ago": 5, 
       "sep": 5,
       "oct": 5, 
       "nov": 4, 
       "dic": 3}

festivo = {"festivo": 0, "no festivo": 1}

dia = {"lun": 0, 
       "mar": 0, 
       "mie": 1, 
       "jue": 1, 
       "vie": 1, 
       "sab": 2, 
       "dom": 2}

meteo = {"nublado": 1, 
         "buen tiempo": 2, 
         "ligeramente mal tiempo": 0}

año = {2018: 0,
       2019: 1}

In [6]:
columnas = ["mes", "festivo", "dia", "meteo", "año"]
mapas = [mes, festivo, dia, meteo, año]
for mapa, col in zip(mapas, columnas):
    df[col] = df[col].map(mapa)

In [7]:
df.sample(2)

,año,mes,festivo,dia,meteo,temperatura,humedad,velo_viento,usu_registrados
596,1,5,1,0,1,0.428758,0.514514,-0.518950,5504
351,0,3,1,0,2,-0.818302,-0.196197,-0.113934,2211


In [8]:
df.to_pickle("../data/bikes_registrados_estand_encod.pkl")

Vamos a probar a guardar también un modelo con encoding y sin estandarizar:

In [9]:
df2 = pd.read_pickle("../data/bikes_limpieza_sin_totales.pkl")
df2.head(2)

,fecha,estacion,año,mes,festivo,dia,laborable,meteo,temperatura,sensacion_termica,humedad,velo_viento,usu_casuales,usu_registrados
0,2018-01-01,invierno,2018,ene,festivo,lun,no laborable,nublado,14.110847,18.18125,80.5833,10.749882,331,654
1,2018-01-02,invierno,2018,ene,no festivo,mar,laborable,nublado,14.902598,17.68695,69.6087,16.652113,131,670


In [10]:
# Vamos a eliminar las variables categóricas "estación" y "laborable"  porque las creamos a partir del resto de variables predictoras y, además, ya comprobamos en la visualización que tenían una relación muy alta con las otras VP.
df2.drop(["laborable", "estacion", "usu_casuales"], axis = 1, inplace= True)

In [11]:
for mapa, col in zip(mapas, columnas):
    df2[col] = df2[col].map(mapa)

df2.sample(2)

,fecha,año,mes,festivo,dia,meteo,temperatura,sensacion_termica,humedad,velo_viento,usu_registrados
655,2019-10-18,1,5,1,1,1,21.4225,25.63125,72.8333,15.874779,6501
12,2018-01-13,0,0,1,2,2,6.7650,7.54415,47.0417,20.167000,1368


In [12]:
df.to_pickle("../data/bikes_registrados_encod_sin_estand.pkl")